In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import geemap
import ee


ee.Authenticate()
ee.Initialize(project='twoearthengineproject')


In [3]:
physio_koshi = gpd.read_file(r"G:\learn\python_works\Work_projects\Koshi_chetan\chetan_cas\works\data\gis\koshi_basin_nepal\koshi_nepal_basin.shp")
whole_koshi_basin = physio_koshi.dissolve()
ee_physio_koshi = ee.FeatureCollection(geemap.geopandas_to_ee(whole_koshi_basin))

In [4]:
# The basin data is already in vector format
basinATLAS = ee.FeatureCollection('WWF/HydroATLAS/v1/Basins/level07')

kosi_region = ee_physio_koshi # Assuming this is a defined ee.Geometry

# Simply clip the vector collection directly to your region of interest
koshi_basins = basinATLAS.filterBounds(kosi_region)



In [22]:
# 1. Define the Point of Interest (Chatara dam)
poi = ee.Geometry.Point([87.165, 26.732])

# 2. Import the HydroATLAS Level-12 Watershed Data
hydroatlas_basins = ee.FeatureCollection("WWF/HydroATLAS/v1/Basins/level12")

# 3. Find the main basin containing the point of interest
downstream_basin = hydroatlas_basins.filterBounds(poi).first()

# 4. Use `ee.List.iterate` to get all upstream basins.
def get_upstream_basins(start_basin):
    # The maximum number of iterations. 100 should be sufficient for most basins.
    # We create a list of numbers to iterate over on the server side.
    max_iterations = 100
    iteration_list = ee.List.sequence(1, max_iterations)

    # Define the initial state (the starting basin)
    initial_collection = ee.FeatureCollection([start_basin])

    # The function to be applied at each iteration.
    def iterate_body(number, current_collection):
        current_collection = ee.FeatureCollection(current_collection)
        
        # Get the IDs of the current basins.
        basin_ids = current_collection.aggregate_array('HYBAS_ID')
        
        # Find all basins whose 'NEXT_DOWN' ID is in our current collection.
        upstream_features = hydroatlas_basins.filter(ee.Filter.inList('NEXT_DOWN', basin_ids))
        
        # Merge the new basins into the collection.
        return current_collection.merge(upstream_features)

    # Run the iterate function and get the final collection.
    result_collection = ee.FeatureCollection(
        iteration_list.iterate(iterate_body, initial_collection)
    )
    
    return result_collection

# Get the final collection of all upstream basins.
koshi_basin_upstream = get_upstream_basins(downstream_basin)

# 5. Visualize the results
Map = geemap.Map(center=[27.5, 87.5], zoom=8)
Map.addLayer(koshi_basin_upstream.style(color='0000FF', fillColor='0000FF33'), {}, 'Koshi Basin (HydroATLAS)')
Map.addLayer(poi, {'color': 'red'}, 'Chatara Dam')
Map.centerObject(koshi_basin_upstream)

# Display the map
Map

EEException: Computation timed out.

In [5]:
# --- Create and Display the Map ---
# Create an interactive map object.
Map = geemap.Map(center=[27.5, 86.8], zoom=8)
# Define a function to generate a text label for each feature.
def add_labels(feature):
    return feature.set({
        'label': ee.String('HYBAS_ID: ').cat(ee.Number(feature.get('HYBAS_ID')).format())
    })

# Map the function over the feature collection to add a 'label' property to each feature.
labeled_basins = koshi_basins.map(add_labels)

# Add the basins to the map with a custom style.
vis_params = {
    'color': '0000FF',
    'fillColor': '0000FF55'
}
Map.addLayer(labeled_basins, vis_params, 'Koshi Basins')

# Add the labels to the map.
# The 'label' property from the previous step will be used for the text.
Map.addLayer(labeled_basins, {}, 'HYBAS_ID Labels', True, 1)

# Center the map on the basins.
Map.centerObject(koshi_basins, 8)

# Display the map.
Map

Map(center=[27.47400223552799, 86.74222028763057], controls=(WidgetControl(options=['position', 'transparent_b…

In [43]:
df = geemap.ee_to_df(koshi_basins)

In [ ]:
print(df)

   COAST  DIST_MAIN  DIST_SINK  ENDO    HYBAS_ID    MAIN_BAS   NEXT_DOWN  \
0      0      757.9      757.9     0  4070875600  4070025450  4070892840   
1      0      521.8      521.8     0  4070911800  4070025450  4070912840   
2      0      668.8      668.8     0  4070892840  4070025450  4070900840   
3      0      577.5      577.5     0  4070891520  4070025450  4070897870   
4      0      519.9      519.9     0  4070905280  4070025450  4070912370   

    NEXT_SINK  ORDER_  PFAF_ID  ...  wet_pc_u02  wet_pc_u03  wet_pc_u04  \
0  4070025450       3  4524255  ...           0           0           0   
1  4070025450       3  4524210  ...           0           0          20   
2  4070025450       3  4524253  ...           0           0           1   
3  4070025450       3  4524169  ...           0           0          19   
4  4070025450       4  4524164  ...           0           0          36   

   wet_pc_u05  wet_pc_u06  wet_pc_u07  wet_pc_u08  wet_pc_u09  wet_pc_ug1  \
0           0  

In [56]:
geojson_filename = 'koshi_basins.geojson'
geemap.ee_to_geojson(
    ee_object=koshi_basins,
    filename=geojson_filename
)

print(f"Export of '{geojson_filename}' successful!")

Export of 'koshi_basins.geojson' successful!


In [59]:
gdf = gpd.read_file('koshi_basins.geojson')
gdf.to_file('koshi_basins.shp')

In [10]:
pip install rabpro

Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following yanked versions: 0.2.1, 0.2.2
ERROR: Could not find a version that satisfies the requirement rabpro (from versions: none)
ERROR: No matching distribution found for rabpro


In [11]:

import ee
import geemap
import rabpro
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'rabpro'